## Training a simple MLP using all the data provided

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re as re
import math

import h5py
import json

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


### 1. Load all files & put into dataframes

Negative training:
- `pfam_training_data_augment.h5`
- `non_cazy_kegg.h5`

Positive training:
- `vicreg_train_val_embeddings_noCAZOME_noLargeSeqs_combined.h5` (+ use `cazy_family_by_taxa_60.json` to pick a representative sample across all enzyme classes)


In [2]:
PUL_embeddings = []
PUL_keys = []
with h5py.File('C:\\Users\\alpha\Documents\\jennifer\\maths\\SRIM\\code\\PUL.h5', 'r') as f:
    for key in f.keys():
        PUL_embeddings.append(np.array(f[key][()]))
        PUL_keys.append(key)
        


In [4]:
PFAM_embeddings = []
PFAM_keys = []
with h5py.File('C:\\Users\\alpha\\Documents\\jennifer\\maths\\SRIM\\code\\pfam_training_data_augment.h5', 'r') as f:
    for key in f.keys():
        PFAM_embeddings.append(f[key][()])
        PFAM_keys.append(key)

In [5]:
kegg_embeddings = []
kegg_keys = []
with h5py.File('C:\\Users\\alpha\\Documents\\jennifer\\maths\\SRIM\\code\\non_cazy_kegg.h5', 'r') as f:
    for key in f.keys():
        kegg_embeddings.append(f[key][()])
        kegg_keys.append(key)

In [6]:
#PUL file:
from Bio import SeqIO
import seaborn as sns
f_path = 'C:\\Users\\alpha\\OneDrive - University of Cambridge\\BACKUP 14-04-22\\docs\\Maths\\SRIM\\code\\PUL.faa'
PUL_array, PUL_keys2 = [], []

with open(f_path, mode='r') as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        identifier, description = record.id, record.description
        PUL_keys2.append(identifier)
        if 'CAZyme' in description:
            PUL_array.append(1)
        else:
            PUL_array.append(0)

PUL_array = np.array(PUL_array)
PUL_array = PUL_array.reshape(np.shape(PUL_array)[0],-1)

col_label=['emb'+str(i) for i in range(len(list(PUL_embeddings)[0]))]
col_label.append('cazy')

PUL_df = pd.DataFrame(data=np.concatenate([PUL_embeddings,PUL_array], axis=1), index=PUL_keys2, columns=col_label)

`PUL_array` stores a 1 if Cazyme; 0 if not

All of the PULs are stored in `PUL_df`, which also indicate if protein is CAZyme (1) or not (0)



In [7]:
#non-cazymes:
non_cazy_embeddings = np.concatenate([PFAM_embeddings,kegg_embeddings], axis=0)
non_cazy_keys = np.concatenate([PFAM_keys, kegg_keys], axis=0)
non_cazy_df = pd.DataFrame(data=non_cazy_embeddings, index=non_cazy_keys, columns=['emb'+str(i) for i in range(len(list(non_cazy_embeddings)[0]))])
non_cazy_df = non_cazy_df.join(pd.DataFrame(data=np.zeros(np.array(non_cazy_keys).shape[0]), index=non_cazy_keys, columns=['cazy']), how='inner')


In [8]:
cazy_embeddings = []
cazy_keys = []
with h5py.File('C:\\Users\\alpha\\Documents\\jennifer\\maths\\SRIM\\code\\vicreg_train_val_embeddings_noCAZOME_noLargeSeqs_combined.h5', 'r') as f:
    for key in f.keys():
        cazy_embeddings.append(f[key][()])
        cazy_keys.append(key)

In [9]:
print(np.shape(PUL_embeddings))
print(np.shape(non_cazy_embeddings))
print(np.shape(cazy_embeddings))

(7699, 1024)
(59673, 1024)
(244592, 1024)


### 2. Create lookup using .json file

In [10]:
family_ids = []
family_keys = []
file = open('C:\\Users\\alpha\\Documents\\jennifer\\maths\\SRIM\\code\\cazy_family_by_taxa_60.json')

In [11]:
# type(file)

In [12]:
cazy_ids = []
cazy_families = []
file = open('C:\\Users\\alpha\\Documents\\jennifer\\maths\\SRIM\\code\\cazy_family_by_taxa_60.json')
data = json.loads(file.read())

for key in data.keys(): #keys are IDs, values are classes
        # print(key)
        cazy_ids.append(key)
        cazy_families.append(data.get(key))


In [13]:
# print(np.shape(cazy_ids))
# print(np.shape(cazy_families))
# type(cazy_ids)
# print(np.shape(np.array(cazy_families)))
concat=np.concatenate((np.reshape(np.array(cazy_ids),(232736,1)), np.reshape(np.array(cazy_families),(232736,1))), axis=1)
# print(concat)
# print(np.shape(concat))

In [14]:
# cazy_df = pd.DataFrame(data=concat , columns=['id','class'])
cazy_df_2 = pd.DataFrame(data=cazy_families, index=cazy_ids , columns=['class'])

Create dataframe of cazymes with extra columnns containing embeddings, then inner join with cazy_df_2 (intersect dataframes at indices)

In [15]:
cazy_df_1 = pd.DataFrame(data=cazy_embeddings, index=cazy_keys, columns=['emb'+str(i) for i in range(len(list(cazy_embeddings)[0]))])

# long (~12 min)

In [16]:
cazy_df = cazy_df_2.join(cazy_df_1, how='inner')
print(cazy_df)

             class      emb0      emb1      emb2      emb3      emb4  \
AZS17016.1  GH13_2  0.049500  0.037415 -0.038025  0.031235  0.017197   
AVO05213.1    GH32 -0.014870  0.036865  0.009872 -0.011703  0.015808   
QPG94344.1    GT61  0.016739 -0.028412  0.026016  0.017227  0.013565   
APD47233.1     GT4  0.015976 -0.009315 -0.032532  0.033173  0.011223   
ANU32067.1     GT4  0.010872 -0.021881 -0.008972 -0.013237  0.000924   
...            ...       ...       ...       ...       ...       ...   
QMI07227.1    GT83 -0.008034 -0.009453  0.029785  0.033478  0.024063   
UMA63307.1     GT2  0.019272 -0.044830  0.024490  0.026031  0.026672   
UPW01296.1     GT4  0.055878 -0.059723 -0.039551 -0.001044  0.057983   
ULL17023.1    GH43  0.013275  0.033997  0.013687  0.021133  0.018799   
USF23742.1     GT9  0.022705 -0.043182 -0.023972  0.007046 -0.025818   

                emb5      emb6      emb7      emb8  ...   emb1014   emb1015  \
AZS17016.1  0.031616 -0.019043 -0.046234  0.007179  ... 

In [17]:
# key_list = list(data.keys())
# val_list = list(data.values())

# ind=0
# for key in data.keys(): #keys are IDs, values are classes
#         cazy_ids.append(key)
#         cazy_families.append(data.get(key))
#         if 'GH' in key:
            

In [18]:
cazy_GH_df = cazy_df[cazy_df['class'].str.contains('GH')]
cazy_GT_df = cazy_df[cazy_df['class'].str.contains('GT')]
cazy_PL_df = cazy_df[cazy_df['class'].str.contains('PL')]
cazy_CE_df = cazy_df[cazy_df['class'].str.contains('CE')]

allclasses=['GH', 'GT', 'PL', 'CE']
cazy_other_df = cazy_df[cazy_df['class'].str.contains('|'.join(allclasses))]

#then use pd.DataFrame.sample to take random sample of items across axis 0


size of first four dataframes add up to 232736; so in fact no other classes to account for

Sample proportionally to size of each class:

In [19]:
x=5000/232736
new_cazy_GH_df = cazy_GH_df.sample(frac=x, axis=0)
new_cazy_GT_df = cazy_GT_df.sample(frac=x, axis=0)
new_cazy_PL_df = cazy_PL_df.sample(frac=x, axis=0)
new_cazy_CE_df = cazy_CE_df.sample(frac=x, axis=0)

Concatenate into one df: `cazy_train_df` contains the positive training data (5001 embeddings):

In [20]:
cazy_train_df = pd.concat([new_cazy_GH_df, new_cazy_GT_df, new_cazy_PL_df, new_cazy_CE_df], axis=0)
cazy_train_df = cazy_train_df.join(pd.DataFrame(data=np.ones(np.array(cazy_ids).shape[0]), index=cazy_ids, columns=['cazy']), how='inner')

In [21]:
cazy_train_df

,class,emb0,emb1,emb2,emb3,emb4,emb5,emb6,emb7,emb8,...,emb1015,emb1016,emb1017,emb1018,emb1019,emb1020,emb1021,emb1022,emb1023,cazy
QOL01086.1,GH31,0.068115,0.009529,0.044281,-0.039459,0.003263,0.009926,-0.074646,-0.043121,-0.054077,...,-0.049164,-0.023560,-0.031158,0.071106,0.025864,-0.001612,0.008064,0.061768,0.048798,1.0
BBG98846.1,GH32,0.030823,0.082581,0.065735,-0.014557,0.005985,0.007996,-0.008774,-0.032104,0.061371,...,0.006680,0.029663,-0.050018,0.052002,0.009552,0.007038,0.036499,0.015991,0.032928,1.0
UOB18734.1,GH29,0.044586,0.096252,0.019272,0.014107,-0.008736,0.069031,-0.054413,-0.088379,0.014938,...,-0.008293,-0.030853,-0.055084,0.054749,0.027939,-0.001834,0.000238,0.035553,0.029160,1.0
BAU42234.1,GH5_39,0.052338,0.069763,0.018265,-0.003042,-0.013245,0.026962,0.006710,-0.076721,0.045898,...,-0.020416,-0.019775,-0.055206,0.014702,0.022125,-0.018082,-0.006020,0.015549,0.081116,1.0
UKZ59741.1,GH16_18,-0.017899,0.019196,0.092224,0.018677,0.010742,-0.032684,-0.027451,-0.080322,-0.059509,...,-0.014549,0.002020,-0.006351,0.072266,-0.025681,0.022720,0.039886,0.019455,0.022537,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AGQ53951.1,CE9,0.064209,0.070435,0.011192,0.029099,-0.021805,0.029556,-0.036926,-0.059082,0.024155,...,-0.012581,-0.001436,-0.052856,0.017151,-0.021179,-0.017365,-0.042297,0.018311,0.012939,1.0
AKK12048.1,CE1,0.056427,-0.032715,-0.001029,-0.026855,-0.015839,0.021698,-0.085632,-0.047546,0.009430,...,-0.056641,-0.033875,-0.043243,-0.002583,0.048615,0.001557,-0.013275,0.016235,0.047058,1.0
ALR14359.1,CE14,-0.002836,-0.002098,-0.003376,0.019119,0.025253,0.026108,-0.002766,-0.076538,0.030563,...,-0.004440,-0.024658,-0.077637,-0.032532,0.003077,-0.020401,-0.058777,0.007858,-0.004440,1.0
UAY05060.1,CE4,0.021118,0.007248,0.033295,-0.013657,0.064392,0.069153,-0.035309,-0.058472,-0.021759,...,-0.005718,0.023819,-0.091919,0.043884,0.018112,-0.016647,-0.064270,0.037750,0.057495,1.0


### 3. Create training dataframe + train a simple MLP

Concatenate non-cazy and cazy training dataframes (size = 10000 + 5001)

In [22]:
non_cazy_train_df = non_cazy_df.sample(n=10000, axis=0)

(a) Use PUL file entirely for inference:

In [23]:
train_df = pd.concat([cazy_train_df, non_cazy_train_df], axis=0).iloc[:,1:] #removes 'class' column

In [24]:
X_train, y_train = np.array(train_df.iloc[:,:-1]), np.array(train_df.iloc[:,-1:])
X_test, y_test = np.array(PUL_df.iloc[:,:-1]), np.array(PUL_df.iloc[:,-1:])

MLP:

In [25]:
mlp = MLPClassifier(hidden_layer_sizes=(100,100),
                     activation = 'logistic',
                     solver = 'adam',
                     verbose = True).fit(X_train,y_train)

c:\Users\alpha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.57923569
Iteration 2, loss = 0.22663311
Iteration 3, loss = 0.09038740
Iteration 4, loss = 0.06436858
Iteration 5, loss = 0.05390895
Iteration 6, loss = 0.04712505
Iteration 7, loss = 0.04249734
Iteration 8, loss = 0.03905014
Iteration 9, loss = 0.03662527
Iteration 10, loss = 0.03443770
Iteration 11, loss = 0.03165569
Iteration 12, loss = 0.02993074
Iteration 13, loss = 0.02831996
Iteration 14, loss = 0.02698151
Iteration 15, loss = 0.02557702
Iteration 16, loss = 0.02877134
Iteration 17, loss = 0.02452250
Iteration 18, loss = 0.02429665
Iteration 19, loss = 0.02276445
Iteration 20, loss = 0.02196946
Iteration 21, loss = 0.02144059
Iteration 22, loss = 0.02069940
Iteration 23, loss = 0.02018774
Iteration 24, loss = 0.01991682
Iteration 25, loss = 0.01897547
Iteration 26, loss = 0.01859272
Iteration 27, loss = 0.01812845
Iteration 28, loss = 0.01739796
Iteration 29, loss = 0.01678970
Iteration 30, loss = 0.01657271
Iteration 31, loss = 0.01653428
Iteration 32, los

In [26]:
y_pred = mlp.predict(X_test)

In [27]:
print(accuracy_score(y_test,y_pred))
#confusion matrix
mat = confusion_matrix(y_test, y_pred)
cfmat_df = pd.DataFrame(np.array(mat))
index_, columns_ = ['Not_cazyme','Cazyme'], ['Pred_not_cazyme', 'Pred_cazyme']
cfmat_df.index, cfmat_df.columns = index_, columns_

print(cfmat_df)

0.6030653331601507
            Pred_not_cazyme  Pred_cazyme
Not_cazyme             3493         2541
Cazyme                  515         1150


- Using PUL file entirely for inference produces terrible results (59.2%); worse than trivial classifier (78.4%) - why??

- positive prediction is poor

- Increasing number of layers gives better results (10 layers: 75.1%) for (a) but worse for (c) (10 layers: 80.8%) (why)








Trivial Classifier:

In [233]:
s=sum(y_test)
p=len(y_test)
#predict all zeros:
trivial_score = (p-s)/p
print('Accuracy of trivial classifier = ',trivial_score)

Accuracy of trivial classifier =  [0.7788961]


(b) Use PUL file (60/40) + `train_df` for training:

In [234]:
train_ind = np.zeros(PUL_df.shape[0])
train_ind[:int(np.floor(PUL_df.shape[0]*0.6))] = 1 #prop. train
np.random.shuffle(train_ind)

X_PUL, y_PUL = PUL_df.to_numpy()[:,:-1], PUL_df.to_numpy()[:,-1:]

X_PUL_train, y_PUL_train = X_PUL[train_ind==1,:], y_PUL[train_ind==1]
X_PUL_test, y_PUL_test = X_PUL[train_ind==0,:], y_PUL[train_ind==0,:]

In [217]:
X1_train, y1_train = np.concatenate([np.array(train_df.iloc[:,:-1]), X_PUL_train]), np.concatenate([np.array(train_df.iloc[:,-1:]), y_PUL_train])
X1_test, y1_test = X_PUL_test, y_PUL_test

In [238]:
mlp1 = MLPClassifier(hidden_layer_sizes=4,
                    activation = 'relu',
                    solver = 'adam',
                    verbose = True).fit(X1_train,y1_train)

c:\Users\alpha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.91555086
Iteration 2, loss = 0.65810590
Iteration 3, loss = 0.52095724
Iteration 4, loss = 0.44739372
Iteration 5, loss = 0.39395954
Iteration 6, loss = 0.35666295
Iteration 7, loss = 0.33171776
Iteration 8, loss = 0.31405388
Iteration 9, loss = 0.30081959
Iteration 10, loss = 0.29050557
Iteration 11, loss = 0.28240896
Iteration 12, loss = 0.27574187
Iteration 13, loss = 0.27004822
Iteration 14, loss = 0.26515674
Iteration 15, loss = 0.26075769
Iteration 16, loss = 0.25674166
Iteration 17, loss = 0.25312778
Iteration 18, loss = 0.24979509
Iteration 19, loss = 0.24666423
Iteration 20, loss = 0.24376843
Iteration 21, loss = 0.24102780
Iteration 22, loss = 0.23841209
Iteration 23, loss = 0.23602843
Iteration 24, loss = 0.23397868
Iteration 25, loss = 0.23163447
Iteration 26, loss = 0.22964655
Iteration 27, loss = 0.22766914
Iteration 28, loss = 0.22584344
Iteration 29, loss = 0.22407316
Iteration 30, loss = 0.22237475
Iteration 31, loss = 0.22083444
Iteration 32, los

c:\Users\alpha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [239]:
y1_pred = mlp1.predict(X1_test)

In [247]:
print(accuracy_score(y1_test,y1_pred))
#confusion matrix
mat = confusion_matrix(y1_test, y1_pred)
cfmat_df = pd.DataFrame(np.array(mat))
index_, columns_ = ['Not_cazyme','Cazyme'], ['Pred_not_cazyme', 'Pred_cazyme']
cfmat_df.index, cfmat_df.columns = index_, columns_

print(cfmat_df)

0.6642857142857143
            Pred_not_cazyme  Pred_cazyme
Not_cazyme             1864          586
Cazyme                  448          182


- 66.4% accuracy; still much worse than trivial classifier

- Positive prediction really poor

(c) Only use PUL file for training and testing: (60/40)

In [243]:
X2_train, y2_train = X_PUL_train, y_PUL_train
X2_test, y2_test = X_PUL_test, y_PUL_test

In [260]:
mlp2 = MLPClassifier(hidden_layer_sizes=3,
                    activation = 'relu',
                    solver = 'adam',
                    verbose = True).fit(X2_train,y2_train)

c:\Users\alpha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.58689011
Iteration 2, loss = 0.52479349
Iteration 3, loss = 0.51000566
Iteration 4, loss = 0.49995585
Iteration 5, loss = 0.49270498
Iteration 6, loss = 0.48712732
Iteration 7, loss = 0.48312494
Iteration 8, loss = 0.48033365
Iteration 9, loss = 0.47768611
Iteration 10, loss = 0.47554593
Iteration 11, loss = 0.47401833
Iteration 12, loss = 0.47267829
Iteration 13, loss = 0.47114940
Iteration 14, loss = 0.46974059
Iteration 15, loss = 0.46905167
Iteration 16, loss = 0.46738163
Iteration 17, loss = 0.46638879
Iteration 18, loss = 0.46511555
Iteration 19, loss = 0.46414475
Iteration 20, loss = 0.46316818
Iteration 21, loss = 0.46200680
Iteration 22, loss = 0.46110203
Iteration 23, loss = 0.46036844
Iteration 24, loss = 0.45954305
Iteration 25, loss = 0.45835839
Iteration 26, loss = 0.45757776
Iteration 27, loss = 0.45684257
Iteration 28, loss = 0.45597062
Iteration 29, loss = 0.45506901
Iteration 30, loss = 0.45409449
Iteration 31, loss = 0.45320411
Iteration 32, los

c:\Users\alpha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [261]:
y2_pred = mlp2.predict(X2_test)

In [262]:
print(accuracy_score(y2_test,y2_pred))
mat = confusion_matrix(y2_test, y2_pred)
cfmat_df = pd.DataFrame(np.array(mat))
index_, columns_ = ['Not_cazyme','Cazyme'], ['Pred_not_cazyme', 'Pred_cazyme']
cfmat_df.index, cfmat_df.columns = index_, columns_

print(cfmat_df)

0.8074675324675324
            Pred_not_cazyme  Pred_cazyme
Not_cazyme             2281          169
Cazyme                  424          206


80.7% accuracy